In [67]:
import pandas as pd
from datetime import datetime

# Load the dataset
data = pd.read_csv("/content/sample_data/train.csv")

In [68]:
# Check the column names
print(data.columns)

Index(['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven',
       'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power',
       'Seats', 'New_Price', 'Price'],
      dtype='object')


In [69]:
# a) Handling missing values
# Look for missing values in all columns
missing_values = data.isnull().sum()

# Decide whether to impute or drop missing values based on the proportion of missing values
# If missing values are less than 5% of the total entries in a column, impute with mean, median, or mode
# Otherwise, drop the column
for col in data.columns:
    if missing_values[col] < 0.05 * len(data):
        if data[col].dtype == 'object':
            mode_val = data[col].mode()[0]
            data[col].fillna(mode_val, inplace=True)
        else:
            mean_val = data[col].mean()
            data[col].fillna(mean_val, inplace=True)
    else:
        data.drop(columns=[col], inplace=True)

# b) Removing units from attributes
data['Mileage'] = data['Mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '')
data['Engine'] = data['Engine'].str.replace(' CC', '')
data['Power'] = data['Power'].str.replace(' bhp', '')

# c) Changing categorical variables into numerical one hot encoded value
data = pd.get_dummies(data, columns=['Fuel_Type', 'Transmission'])

# d) Creating a new feature - Current Age of the car
current_year = datetime.now().year
data['Current_Age'] = current_year - data['Year']

# e) Performing select, filter, rename, mutate, arrange, and summarize operations
# Select operation - selecting specific columns
selected_data = data[['Name', 'Location', 'Year', 'Mileage', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'Transmission_Automatic', 'Transmission_Manual', 'Price']]

# Filter operation - filtering cars with price greater than 50
filtered_data = data[data['Price'] > 50]

# Rename operation - renaming columns
renamed_data = data.rename(columns={'Year': 'Manufacture_Year', 'Price': 'Price_in_Lakhs'})

# Mutate operation - adding a new column for the value of 'Engine + Power'
data['Engine_Power'] = data['Engine'] + data['Power']

# Arrange operation - sorting data based on 'Current_Age' in descending order
arranged_data = data.sort_values(by='Current_Age', ascending=False)

# Summarize with group by operation - calculating mean price for each fuel type
summary_data = data.groupby('Fuel_Type_Petrol')['Price'].mean().reset_index()

# Displaying the summary statistics
print("Summary Statistics:")
print(summary_data)

# Displaying the modified dataset
print("Modified Dataset:")
print(data.head(15))

Summary Statistics:
   Fuel_Type_Petrol      Price
0                 0  12.960632
1                 1   5.756688
Modified Dataset:
    Unnamed: 0                                 Name    Location  Year  \
0            1     Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1            2                         Honda Jazz V     Chennai  2011   
2            3                    Maruti Ertiga VDI     Chennai  2012   
3            4      Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4            6               Nissan Micra Diesel XV      Jaipur  2013   
5            7    Toyota Innova Crysta 2.8 GX AT 8S      Mumbai  2016   
6            8  Volkswagen Vento Diesel Comfortline        Pune  2013   
7            9       Tata Indica Vista Quadrajet LS     Chennai  2012   
8           10                     Maruti Ciaz Zeta       Kochi  2018   
9           11          Honda City 1.5 V AT Sunroof     Kolkata  2012   
10          12                Maruti Swift VDI BSIV      Jaipur  2